In [ ]:
## This file implements neural networks before and after lasso selection for p11kpresabs_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p11kpresabs_qual.csv')
df.shape

(253, 822)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      1
2      2
3      2
4      2
      ..
248    2
249    1
250    2
251    2
252    2
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,...,ACAAGTCGCTGAAATATT,ACAAACTTTCTAGGTT,AATCACCCCTT,AAGGGGTGATTT,AAGGGGTGATTTT,AAGATGATTTATCCAACTTT,AACTTTCTAGGTT,AACCTAGAAAGTTT,AACATCTTTTATTT,pheno
0,107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,109,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,115,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,2
3,120335,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,2
4,120337,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,2


In [6]:
df['pheno'].value_counts()

2    181
1     47
0     25
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 821)

In [9]:
df_clean.head()

,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,TTTTTAGGTAAGG,...,ACAAGTCGCTGAAATATT,ACAAACTTTCTAGGTT,AATCACCCCTT,AAGGGGTGATTT,AAGGGGTGATTTT,AAGATGATTTATCCAACTTT,AACTTTCTAGGTT,AACCTAGAAAGTTT,AACATCTTTTATTT,pheno
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,2
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,2
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,2


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 821) (253,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 181), (1, 181), (2, 181)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [12]:
############# Fully-Connected Neural Network ################

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [16]:
dat

,0,test
0,SR1129,2
1,NRS185,2
2,NRS243,1
3,BCH-SA-04,0
4,504,1
...,...,...
158,CFBREBSa131,2
159,CFBREBSa133,1
160,NRS256,2
161,GA48963,1


In [17]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 370us/step - loss: 1.2958 - accuracy: 0.3842 - val_loss: 1.1623 - val_accuracy: 0.3497
Epoch 2/100
380/380 [==============================] - 0s 500us/step - loss: 1.0366 - accuracy: 0.4711 - val_loss: 1.0661 - val_accuracy: 0.4479
Epoch 3/100
380/380 [==============================] - 0s 310us/step - loss: 0.9914 - accuracy: 0.5263 - val_loss: 0.9755 - val_accuracy: 0.5706
Epoch 4/100
380/380 [==============================] - 0s 648us/step - loss: 0.8832 - accuracy: 0.6158 - val_loss: 0.9160 - val_accuracy: 0.5276
Epoch 5/100
380/380 [==============================] - 0s 654us/step - loss: 0.8389 - accuracy: 0.6079 - val_loss: 0.8976 - val_accuracy: 0.6135
Epoch 6/100
380/380 [==============================] - 0s 279us/step - loss: 0.8161 - accuracy: 0.6184 - val_loss: 0.9170 - val_accuracy: 0.6074
Epoch 7/100
380/380 [==============================] - 0s 275us/step - loss: 0.7946 

Epoch 57/100
380/380 [==============================] - 0s 688us/step - loss: 0.3541 - accuracy: 0.8526 - val_loss: 0.6356 - val_accuracy: 0.7546
Epoch 58/100
380/380 [==============================] - 0s 491us/step - loss: 0.2998 - accuracy: 0.8816 - val_loss: 0.5790 - val_accuracy: 0.7607
Epoch 59/100
380/380 [==============================] - 0s 499us/step - loss: 0.3098 - accuracy: 0.8816 - val_loss: 0.5920 - val_accuracy: 0.7546
Epoch 60/100
380/380 [==============================] - 0s 477us/step - loss: 0.3332 - accuracy: 0.8605 - val_loss: 0.6100 - val_accuracy: 0.7239
Epoch 61/100
380/380 [==============================] - 0s 481us/step - loss: 0.3297 - accuracy: 0.8474 - val_loss: 0.6261 - val_accuracy: 0.7239
Epoch 62/100
380/380 [==============================] - 0s 305us/step - loss: 0.3217 - accuracy: 0.8737 - val_loss: 0.6523 - val_accuracy: 0.7117
Epoch 63/100
380/380 [==============================] - 0s 686us/step - loss: 0.3312 - accuracy: 0.8605 - val_loss: 0.6784 -

In [37]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

163/163 [==============================] - 0s 85us/step
over-sampling test accuracy: 78.53%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([0, 2, 1, 0, 1, 0, 0, 0, 1, 1, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 0,
       0, 2, 0, 0, 0, 1, 0, 0, 2, 2, 1, 0, 1, 0, 1, 1, 1, 1, 0, 2, 0, 2,
       1, 1, 2, 1, 0, 0, 0, 0, 1, 1, 1, 2, 1, 2, 0, 0, 0, 1, 0, 0, 1, 2,
       0, 0, 1, 0, 0, 1, 0, 1, 2, 1, 1, 0, 0, 0, 1, 1, 2, 1, 1, 0, 0, 1,
       1, 2, 0, 0, 1, 0, 1, 2, 2, 0, 2, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0,
       2, 1, 0, 1, 2, 0, 0, 1, 1, 0, 0, 2, 1, 2, 0, 2, 2, 0, 2, 0, 0, 1,
       2, 1, 1, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 1, 0, 2, 0, 0, 0, 0,
       2, 0, 0, 2, 0, 1, 2, 1, 0])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,SR1129,2,0
1,NRS185,2,2
2,NRS243,1,1
3,BCH-SA-04,0,0
4,504,1,1
...,...,...,...
158,CFBREBSa131,2,0
159,CFBREBSa133,1,1
160,NRS256,2,2
161,GA48963,1,1


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,0.991729,0.001580,0.006691
1,0.011410,0.104540,0.884050
2,0.000098,0.985774,0.014128
3,0.995715,0.000013,0.004272
4,0.113516,0.788538,0.097947
...,...,...,...
158,0.883826,0.000551,0.115623
159,0.000303,0.661321,0.338375
160,0.000027,0.007202,0.992771
161,0.000209,0.844463,0.155327


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p11.csv", index = False,
         header=None)

In [41]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 183us/step - loss: 0.2487 - accuracy: 0.9026 - val_loss: 0.5889 - val_accuracy: 0.7791
Epoch 2/100
380/380 [==============================] - 0s 160us/step - loss: 0.2469 - accuracy: 0.8816 - val_loss: 0.6103 - val_accuracy: 0.7362
Epoch 3/100
380/380 [==============================] - 0s 180us/step - loss: 0.2691 - accuracy: 0.8816 - val_loss: 0.6027 - val_accuracy: 0.7853
Epoch 4/100
380/380 [==============================] - 0s 165us/step - loss: 0.2568 - accuracy: 0.8921 - val_loss: 0.5829 - val_accuracy: 0.7853
Epoch 5/100
380/380 [==============================] - 0s 158us/step - loss: 0.2419 - accuracy: 0.9000 - val_loss: 0.6354 - val_accuracy: 0.7853
Epoch 6/100
380/380 [==============================] - 0s 149us/step - loss: 0.2641 - accuracy: 0.8868 - val_loss: 0.6748 - val_accuracy: 0.7362
Epoch 7/100
380/380 [==============================] - 0s 134us/step - loss: 0.2567 

Epoch 57/100
380/380 [==============================] - 0s 110us/step - loss: 0.2208 - accuracy: 0.9026 - val_loss: 0.6264 - val_accuracy: 0.7853
Epoch 58/100
380/380 [==============================] - 0s 111us/step - loss: 0.2047 - accuracy: 0.9211 - val_loss: 0.6408 - val_accuracy: 0.7730
Epoch 59/100
380/380 [==============================] - 0s 359us/step - loss: 0.2110 - accuracy: 0.9053 - val_loss: 0.6313 - val_accuracy: 0.8098
Epoch 60/100
380/380 [==============================] - 0s 165us/step - loss: 0.2153 - accuracy: 0.9079 - val_loss: 0.6313 - val_accuracy: 0.8098
Epoch 61/100
380/380 [==============================] - 0s 132us/step - loss: 0.2150 - accuracy: 0.9105 - val_loss: 0.5999 - val_accuracy: 0.7914
Epoch 62/100
380/380 [==============================] - 0s 134us/step - loss: 0.2003 - accuracy: 0.9105 - val_loss: 0.5945 - val_accuracy: 0.7914
Epoch 63/100
380/380 [==============================] - 0s 124us/step - loss: 0.2108 - accuracy: 0.9132 - val_loss: 0.7039 -

In [42]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 90.24%


In [18]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [19]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [20]:
y_prob = df_proba[df_proba['phage']=='p11kpresabs_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[9.91729100e-01, 1.57970380e-03, 6.69116500e-03],
       [1.14097060e-02, 1.04540370e-01, 8.84049900e-01],
       [9.78376850e-05, 9.85774040e-01, 1.41280090e-02],
       [9.95715100e-01, 1.27100590e-05, 4.27222950e-03],
       [1.13515510e-01, 7.88537500e-01, 9.79469700e-02],
       [4.39802830e-01, 4.07325300e-01, 1.52871890e-01],
       [9.97045930e-01, 2.13479500e-03, 8.19317700e-04],
       [9.72656700e-01, 4.84011600e-04, 2.68592800e-02],
       [3.43547600e-02, 9.57373400e-01, 8.27184300e-03],
       [4.14786630e-04, 9.92141370e-01, 7.44387800e-03],
       [4.84715070e-02, 1.54534520e-01, 7.96994000e-01],
       [9.88449160e-01, 9.83262600e-03, 1.71824530e-03],
       [4.95854350e-01, 1.48356170e-01, 3.55789540e-01],
       [3.06200950e-01, 3.41136840e-01, 3.52662270e-01],
       [9.88449160e-01, 9.83262600e-03, 1.71824530e-03],
       [9.96626260e-01, 1.57607890e-03, 1.79763920e-03],
       [1.51535050e-01, 1.46507320e-01, 7.01957640e-01],
       [4.47552920e-01, 1.16874

In [15]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [22]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.9317172746836354

In [16]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [24]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.9317172746836354

In [25]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [26]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [27]:
dat2

,0,test
0,NRS027,0
1,CFBRSa07,0
2,CFBRSa27,1
3,504,1
4,CFBREBSa129,0
...,...,...
158,SR3569,2
159,NRS243,1
160,GA48963,1
161,504,1


In [28]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [31]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 660us/step - loss: 1.0803 - accuracy: 0.4132 - val_loss: 1.0392 - val_accuracy: 0.4540
Epoch 2/100
380/380 [==============================] - 0s 319us/step - loss: 1.0433 - accuracy: 0.4711 - val_loss: 1.0081 - val_accuracy: 0.4417
Epoch 3/100
380/380 [==============================] - 0s 244us/step - loss: 0.9865 - accuracy: 0.5342 - val_loss: 0.9426 - val_accuracy: 0.5951
Epoch 4/100
380/380 [==============================] - 0s 301us/step - loss: 0.9300 - accuracy: 0.6079 - val_loss: 0.8803 - val_accuracy: 0.6196
Epoch 5/100
380/380 [==============================] - 0s 283us/step - loss: 0.8783 - accuracy: 0.5921 - val_loss: 0.8754 - val_accuracy: 0.6258
Epoch 6/100
380/380 [==============================] - 0s 157us/step - loss: 0.8566 - accuracy: 0.6474 - val_loss: 0.8344 - val_accuracy: 0.6442
Epoch 7/100
380/380 [==============================] - 0s 184us/step - loss: 0.7801 

In [62]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

163/163 [==============================] - 0s 98us/step
over-sampling test accuracy: 84.05%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([0, 0, 2, 1, 0, 2, 0, 1, 2, 1, 0, 0, 1, 2, 1, 2, 1, 1, 2, 2, 1, 0,
       1, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 2, 1,
       1, 0, 0, 1, 1, 0, 2, 2, 0, 1, 0, 0, 2, 0, 0, 0, 2, 1, 2, 0, 2, 1,
       0, 2, 1, 2, 2, 0, 1, 2, 1, 1, 0, 0, 0, 0, 1, 2, 1, 0, 1, 1, 2, 2,
       0, 0, 2, 0, 1, 0, 0, 1, 0, 2, 2, 0, 0, 0, 1, 0, 1, 2, 0, 2, 1, 0,
       1, 1, 0, 0, 2, 0, 2, 1, 1, 0, 2, 1, 2, 2, 1, 0, 1, 0, 0, 2, 0, 1,
       2, 0, 1, 2, 1, 2, 2, 0, 1, 2, 2, 2, 0, 2, 0, 0, 2, 2, 1, 0, 0, 2,
       1, 1, 2, 2, 2, 1, 1, 1, 0])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,NRS027,0,0
1,CFBRSa07,0,0
2,CFBRSa27,1,2
3,504,1,1
4,CFBREBSa129,0,0
...,...,...,...
158,SR3569,2,2
159,NRS243,1,1
160,GA48963,1,1
161,504,1,1


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,9.988309e-01,0.000262,0.000907
1,9.803634e-01,0.000096,0.019541
2,1.662070e-01,0.187860,0.645933
3,1.041796e-01,0.556848,0.338973
4,9.980018e-01,0.000789,0.001210
...,...,...,...
158,2.914282e-02,0.151264,0.819593
159,8.927739e-09,0.998998,0.001002
160,2.442053e-04,0.959986,0.039770
161,1.041796e-01,0.556848,0.338973


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p11.csv", index = False,
         header=None)

In [66]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 167us/step - loss: 0.2394 - accuracy: 0.8868 - val_loss: 0.4799 - val_accuracy: 0.7914
Epoch 2/100
380/380 [==============================] - 0s 156us/step - loss: 0.2582 - accuracy: 0.8895 - val_loss: 0.3973 - val_accuracy: 0.8282
Epoch 3/100
380/380 [==============================] - 0s 163us/step - loss: 0.2531 - accuracy: 0.8763 - val_loss: 0.4499 - val_accuracy: 0.8160
Epoch 4/100
380/380 [==============================] - 0s 146us/step - loss: 0.2202 - accuracy: 0.9079 - val_loss: 0.4652 - val_accuracy: 0.8528
Epoch 5/100
380/380 [==============================] - 0s 129us/step - loss: 0.2367 - accuracy: 0.8947 - val_loss: 0.5372 - val_accuracy: 0.7853
Epoch 6/100
380/380 [==============================] - 0s 124us/step - loss: 0.2371 - accuracy: 0.8947 - val_loss: 0.4480 - val_accuracy: 0.8466
Epoch 7/100
380/380 [==============================] - 0s 132us/step - loss: 0.2126 

Epoch 57/100
380/380 [==============================] - 0s 209us/step - loss: 0.2115 - accuracy: 0.9053 - val_loss: 0.4603 - val_accuracy: 0.8589
Epoch 58/100
380/380 [==============================] - 0s 145us/step - loss: 0.2130 - accuracy: 0.9026 - val_loss: 0.6357 - val_accuracy: 0.7853
Epoch 59/100
380/380 [==============================] - 0s 158us/step - loss: 0.2138 - accuracy: 0.8974 - val_loss: 0.6300 - val_accuracy: 0.7853
Epoch 60/100
380/380 [==============================] - 0s 140us/step - loss: 0.2422 - accuracy: 0.8947 - val_loss: 0.5397 - val_accuracy: 0.8160
Epoch 61/100
380/380 [==============================] - 0s 125us/step - loss: 0.2665 - accuracy: 0.8737 - val_loss: 0.5049 - val_accuracy: 0.8405
Epoch 62/100
380/380 [==============================] - 0s 119us/step - loss: 0.2467 - accuracy: 0.8842 - val_loss: 0.4940 - val_accuracy: 0.8405
Epoch 63/100
380/380 [==============================] - 0s 114us/step - loss: 0.2294 - accuracy: 0.9053 - val_loss: 0.5363 -

In [67]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 89.66%


In [29]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [30]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [31]:
y_prob2 = df_proba2[df_proba2['phage']=='p11kpresabs_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[9.98830850e-01, 2.62073150e-04, 9.07041700e-04],
       [9.80363400e-01, 9.58994000e-05, 1.95406820e-02],
       [1.66207020e-01, 1.87859740e-01, 6.45933300e-01],
       [1.04179610e-01, 5.56847600e-01, 3.38972750e-01],
       [9.98001750e-01, 7.88705100e-04, 1.20958580e-03],
       [4.54445150e-01, 4.22900280e-04, 5.45132000e-01],
       [9.98001750e-01, 7.88705100e-04, 1.20958580e-03],
       [2.70596900e-03, 9.84363260e-01, 1.29307850e-02],
       [1.66207020e-01, 1.87859740e-01, 6.45933300e-01],
       [2.03490440e-05, 9.97040570e-01, 2.93912880e-03],
       [5.37244700e-01, 6.89718700e-03, 4.55858100e-01],
       [9.98131100e-01, 1.85813630e-03, 1.07736480e-05],
       [3.17332680e-04, 9.09443140e-01, 9.02395700e-02],
       [1.18529715e-01, 3.15663730e-01, 5.65806600e-01],
       [1.13068445e-05, 9.97145100e-01, 2.84360500e-03],
       [1.62177330e-04, 4.50017250e-04, 9.99387740e-01],
       [8.73137200e-12, 9.96745100e-01, 3.25487440e-03],
       [6.01116900e-05, 8.42397

In [32]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9459127462185566

In [33]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9459127462185566

In [34]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [35]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [36]:
dat3

,0,test
0,NRS149,0
1,EUH13,0
2,NRS106,2
3,NRS214,1
4,CFBREBSa129,0
...,...,...
158,NRS027,0
159,CFBRSa70,2
160,CFBREBSa130,0
161,NRS214,1


In [37]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [44]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 541us/step - loss: 1.1313 - accuracy: 0.3605 - val_loss: 1.0493 - val_accuracy: 0.4663
Epoch 2/100
380/380 [==============================] - 0s 158us/step - loss: 1.0639 - accuracy: 0.4737 - val_loss: 1.0293 - val_accuracy: 0.4969
Epoch 3/100
380/380 [==============================] - 0s 199us/step - loss: 0.9962 - accuracy: 0.5105 - val_loss: 0.9547 - val_accuracy: 0.5828
Epoch 4/100
380/380 [==============================] - 0s 162us/step - loss: 0.9509 - accuracy: 0.5789 - val_loss: 0.9205 - val_accuracy: 0.5767
Epoch 5/100
380/380 [==============================] - 0s 151us/step - loss: 0.9030 - accuracy: 0.5974 - val_loss: 0.8759 - val_accuracy: 0.6196
Epoch 6/100
380/380 [==============================] - 0s 188us/step - loss: 0.8588 - accuracy: 0.6053 - val_loss: 0.8588 - val_accuracy: 0.6564
Epoch 7/100
380/380 [==============================] - 0s 168us/step - loss: 0.8288 

In [91]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

163/163 [==============================] - 0s 102us/step
over-sampling test accuracy: 79.75%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([0, 0, 2, 1, 0, 1, 0, 1, 0, 0, 1, 2, 1, 0, 0, 1, 2, 1, 0, 2, 2, 2,
       0, 2, 2, 1, 2, 2, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 0, 1, 1, 1,
       1, 2, 2, 1, 0, 1, 2, 0, 0, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 2, 0, 0, 0, 2, 2, 2, 2, 0, 0, 1,
       2, 0, 2, 0, 0, 0, 1, 2, 1, 1, 0, 1, 2, 0, 1, 1, 2, 0, 2, 1, 1, 1,
       1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0,
       1, 2, 0, 2, 0, 0, 0, 2, 1, 0, 1, 0, 1, 1, 0, 0, 0, 2, 0, 2, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 1])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,NRS149,0,0
1,EUH13,0,0
2,NRS106,2,2
3,NRS214,1,1
4,CFBREBSa129,0,0
...,...,...,...
158,NRS027,0,0
159,CFBRSa70,2,1
160,CFBREBSa130,0,0
161,NRS214,1,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,0.981236,0.001220,0.017544
1,0.962963,0.026475,0.010562
2,0.081841,0.181278,0.736881
3,0.002954,0.968637,0.028409
4,0.993843,0.001009,0.005148
...,...,...,...
158,0.994845,0.001055,0.004101
159,0.049889,0.561527,0.388585
160,0.604731,0.000340,0.394930
161,0.002954,0.968637,0.028409


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p11.csv", index = False,
         header=None)

In [95]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 141us/step - loss: 0.2428 - accuracy: 0.9026 - val_loss: 0.5395 - val_accuracy: 0.7730
Epoch 2/100
380/380 [==============================] - 0s 145us/step - loss: 0.2454 - accuracy: 0.9079 - val_loss: 0.5404 - val_accuracy: 0.8037
Epoch 3/100
380/380 [==============================] - 0s 125us/step - loss: 0.2870 - accuracy: 0.8711 - val_loss: 0.4782 - val_accuracy: 0.8282
Epoch 4/100
380/380 [==============================] - 0s 115us/step - loss: 0.2745 - accuracy: 0.9000 - val_loss: 0.7275 - val_accuracy: 0.7607
Epoch 5/100
380/380 [==============================] - 0s 100us/step - loss: 0.2437 - accuracy: 0.9079 - val_loss: 0.4881 - val_accuracy: 0.8160
Epoch 6/100
380/380 [==============================] - 0s 128us/step - loss: 0.2688 - accuracy: 0.8895 - val_loss: 0.5512 - val_accuracy: 0.8221
Epoch 7/100
380/380 [==============================] - 0s 118us/step - loss: 0.2500 

Epoch 57/100
380/380 [==============================] - 0s 113us/step - loss: 0.2014 - accuracy: 0.9237 - val_loss: 0.5644 - val_accuracy: 0.8160
Epoch 58/100
380/380 [==============================] - 0s 107us/step - loss: 0.1995 - accuracy: 0.9158 - val_loss: 0.4985 - val_accuracy: 0.8405
Epoch 59/100
380/380 [==============================] - 0s 208us/step - loss: 0.2088 - accuracy: 0.9158 - val_loss: 0.5300 - val_accuracy: 0.8160
Epoch 60/100
380/380 [==============================] - 0s 277us/step - loss: 0.1820 - accuracy: 0.9289 - val_loss: 0.5319 - val_accuracy: 0.8221
Epoch 61/100
380/380 [==============================] - 0s 174us/step - loss: 0.1868 - accuracy: 0.9184 - val_loss: 0.5584 - val_accuracy: 0.8160
Epoch 62/100
380/380 [==============================] - 0s 140us/step - loss: 0.1852 - accuracy: 0.9263 - val_loss: 0.6746 - val_accuracy: 0.7914
Epoch 63/100
380/380 [==============================] - 0s 134us/step - loss: 0.2174 - accuracy: 0.9053 - val_loss: 0.4792 -

In [96]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 91.06%


In [38]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [39]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [40]:
y_prob3 = df_proba3[df_proba3['phage']=='p11kpresabs_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[9.81236200e-01, 1.22012330e-03, 1.75436550e-02],
       [9.62963040e-01, 2.64746080e-02, 1.05623360e-02],
       [8.18407300e-02, 1.81278270e-01, 7.36881000e-01],
       [2.95371170e-03, 9.68637300e-01, 2.84089970e-02],
       [9.93843440e-01, 1.00909660e-03, 5.14757540e-03],
       [7.45568350e-02, 5.57530100e-01, 3.67913040e-01],
       [9.94844700e-01, 1.05456050e-03, 4.10082750e-03],
       [3.73867120e-01, 4.66225770e-01, 1.59907090e-01],
       [9.85071540e-01, 3.30949840e-04, 1.45975140e-02],
       [8.27075360e-01, 1.41998840e-01, 3.09256760e-02],
       [9.76711700e-05, 9.79777400e-01, 2.01249220e-02],
       [3.42044800e-03, 3.69428730e-02, 9.59636600e-01],
       [8.05964600e-06, 9.47391100e-01, 5.26008460e-02],
       [5.21736600e-01, 6.87568750e-05, 4.78194650e-01],
       [9.97449100e-01, 2.44725570e-03, 1.03546710e-04],
       [3.02320430e-04, 7.54676160e-01, 2.45021580e-01],
       [2.14490390e-02, 2.85164740e-01, 6.93386200e-01],
       [3.73867120e-01, 4.66225

In [41]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9394065527857577

In [42]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9394065527857577

In [43]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [44]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [45]:
dat4

,0,test
0,SR2852,2
1,CFBREBSa138,0
2,BCH-SA-12,0
3,EUH13,0
4,EUH13,0
...,...,...
158,NRS036,1
159,CA105,1
160,CFBRSa51,1
161,NRS102,1


In [46]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [57]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 744us/step - loss: 1.1390 - accuracy: 0.3684 - val_loss: 1.0621 - val_accuracy: 0.4847
Epoch 2/100
380/380 [==============================] - 0s 184us/step - loss: 1.0336 - accuracy: 0.4658 - val_loss: 1.0125 - val_accuracy: 0.5644
Epoch 3/100
380/380 [==============================] - 0s 157us/step - loss: 0.9844 - accuracy: 0.5711 - val_loss: 0.9786 - val_accuracy: 0.5828
Epoch 4/100
380/380 [==============================] - 0s 186us/step - loss: 0.9369 - accuracy: 0.6158 - val_loss: 0.9575 - val_accuracy: 0.5890
Epoch 5/100
380/380 [==============================] - 0s 200us/step - loss: 0.8896 - accuracy: 0.6158 - val_loss: 0.9384 - val_accuracy: 0.5583
Epoch 6/100
380/380 [==============================] - 0s 220us/step - loss: 0.8499 - accuracy: 0.6763 - val_loss: 0.8763 - val_accuracy: 0.6135
Epoch 7/100
380/380 [==============================] - 0s 213us/step - loss: 0.8050 

Epoch 57/100
380/380 [==============================] - 0s 292us/step - loss: 0.2889 - accuracy: 0.8658 - val_loss: 0.5009 - val_accuracy: 0.7914
Epoch 58/100
380/380 [==============================] - 0s 118us/step - loss: 0.2783 - accuracy: 0.8868 - val_loss: 0.4951 - val_accuracy: 0.8037
Epoch 59/100
380/380 [==============================] - 0s 123us/step - loss: 0.2764 - accuracy: 0.9026 - val_loss: 0.4766 - val_accuracy: 0.7791
Epoch 60/100
380/380 [==============================] - 0s 112us/step - loss: 0.2723 - accuracy: 0.8658 - val_loss: 0.4632 - val_accuracy: 0.7791
Epoch 61/100
380/380 [==============================] - 0s 116us/step - loss: 0.2678 - accuracy: 0.8895 - val_loss: 0.4743 - val_accuracy: 0.8098
Epoch 62/100
380/380 [==============================] - 0s 171us/step - loss: 0.2639 - accuracy: 0.8868 - val_loss: 0.5162 - val_accuracy: 0.7730
Epoch 63/100
380/380 [==============================] - 0s 170us/step - loss: 0.2671 - accuracy: 0.8789 - val_loss: 0.5445 -

In [116]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

163/163 [==============================] - 0s 105us/step
over-sampling test accuracy: 80.37%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([2, 0, 0, 0, 0, 1, 2, 2, 0, 0, 1, 1, 2, 0, 1, 0, 0, 0, 1, 0, 2, 2,
       0, 1, 1, 1, 1, 0, 2, 2, 0, 2, 1, 1, 2, 1, 0, 2, 0, 0, 0, 1, 1, 1,
       0, 1, 2, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 2, 2, 0, 1, 0, 0, 1, 2, 1, 1, 1, 2, 0, 0, 2, 1, 2, 1, 1, 1,
       0, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 2, 2, 2, 2, 1, 0,
       2, 0, 1, 1, 0, 1, 0, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 0, 1, 1,
       1, 1, 2, 1, 0, 0, 1, 1, 0, 0, 2, 0, 2, 2, 1, 1, 1, 0, 0, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,SR2852,2,2
1,CFBREBSa138,0,0
2,BCH-SA-12,0,0
3,EUH13,0,0
4,EUH13,0,0
...,...,...,...
158,NRS036,1,1
159,CA105,1,1
160,CFBRSa51,1,1
161,NRS102,1,1


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,0.000036,0.025301,0.974663
1,0.998706,0.000814,0.000480
2,0.976673,0.004427,0.018900
3,0.919572,0.067330,0.013098
4,0.919572,0.067330,0.013098
...,...,...,...
158,0.000499,0.923615,0.075886
159,0.000005,0.999085,0.000910
160,0.000258,0.999683,0.000059
161,0.000014,0.999135,0.000851


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p11.csv", index = False,
         header=None)

In [120]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 181us/step - loss: 0.2402 - accuracy: 0.8895 - val_loss: 0.5029 - val_accuracy: 0.8282
Epoch 2/100
380/380 [==============================] - 0s 154us/step - loss: 0.2139 - accuracy: 0.9158 - val_loss: 0.5459 - val_accuracy: 0.8221
Epoch 3/100
380/380 [==============================] - 0s 149us/step - loss: 0.2117 - accuracy: 0.9132 - val_loss: 0.5106 - val_accuracy: 0.8160
Epoch 4/100
380/380 [==============================] - 0s 122us/step - loss: 0.2173 - accuracy: 0.8947 - val_loss: 0.4998 - val_accuracy: 0.7914
Epoch 5/100
380/380 [==============================] - 0s 120us/step - loss: 0.2538 - accuracy: 0.8684 - val_loss: 0.4831 - val_accuracy: 0.8037
Epoch 6/100
380/380 [==============================] - 0s 123us/step - loss: 0.2280 - accuracy: 0.9184 - val_loss: 0.4708 - val_accuracy: 0.8344
Epoch 7/100
380/380 [==============================] - 0s 121us/step - loss: 0.3265 

Epoch 57/100
380/380 [==============================] - 0s 177us/step - loss: 0.2011 - accuracy: 0.9158 - val_loss: 0.5217 - val_accuracy: 0.7975
Epoch 58/100
380/380 [==============================] - 0s 185us/step - loss: 0.2038 - accuracy: 0.9105 - val_loss: 0.5321 - val_accuracy: 0.8037
Epoch 59/100
380/380 [==============================] - 0s 167us/step - loss: 0.2410 - accuracy: 0.8842 - val_loss: 0.5548 - val_accuracy: 0.8160
Epoch 60/100
380/380 [==============================] - 0s 189us/step - loss: 0.1935 - accuracy: 0.9237 - val_loss: 0.5489 - val_accuracy: 0.8282
Epoch 61/100
380/380 [==============================] - 0s 181us/step - loss: 0.1891 - accuracy: 0.9184 - val_loss: 0.5409 - val_accuracy: 0.8098
Epoch 62/100
380/380 [==============================] - 0s 220us/step - loss: 0.1942 - accuracy: 0.9158 - val_loss: 0.5760 - val_accuracy: 0.8098
Epoch 63/100
380/380 [==============================] - 0s 192us/step - loss: 0.1969 - accuracy: 0.9000 - val_loss: 0.5637 -

In [121]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 90.78%


In [47]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [48]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [49]:
y_prob4 = df_proba4[df_proba4['phage']=='p11kpresabs_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[3.5529585e-05, 2.5301340e-02, 9.7466314e-01],
       [9.9870634e-01, 8.1354770e-04, 4.8010462e-04],
       [9.7667330e-01, 4.4271164e-03, 1.8899648e-02],
       [9.1957180e-01, 6.7330010e-02, 1.3098269e-02],
       [9.1957180e-01, 6.7330010e-02, 1.3098269e-02],
       [9.3829454e-05, 9.7523670e-01, 2.4669446e-02],
       [2.2173378e-01, 1.8028943e-01, 5.9797674e-01],
       [1.0273278e-05, 2.1151037e-04, 9.9977820e-01],
       [7.4110700e-01, 2.3698530e-01, 2.1907752e-02],
       [9.9330360e-01, 5.6237000e-03, 1.0727432e-03],
       [7.6339087e-03, 9.1458905e-01, 7.7777030e-02],
       [2.9734123e-01, 6.2499960e-01, 7.7659205e-02],
       [5.3477073e-03, 8.6433790e-02, 9.0821856e-01],
       [7.4110700e-01, 2.3698530e-01, 2.1907752e-02],
       [1.7836774e-03, 5.4667970e-01, 4.5153670e-01],
       [9.8375630e-01, 7.4541410e-05, 1.6169135e-02],
       [7.4110700e-01, 2.3698530e-01, 2.1907752e-02],
       [9.9914120e-01, 8.0202933e-04, 5.6638106e-05],
       [4.0357750e-03, 6.102

In [50]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9480261328885181

In [51]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9480261328885181

In [52]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9412656766441169

In [53]:
np.std(ovos)

0.006362533807018114

In [54]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9412656766441169

In [55]:
np.std(ovrs)

0.006362533807018114

In [122]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [123]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 80.67%


In [124]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.020577324900535585


In [125]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [126]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 90.43%


In [127]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.0053634555


In [49]:
############ Feature selection using lasso ##########

In [17]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [18]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X_over[:,1:], y_over)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [19]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [20]:
X_train_features_over = np.vstack((names, X_over[:,1:]))
X_train_features_over = pd.DataFrame(X_train_features_over)

In [21]:
sel_features = X_train_features_over.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_over.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 820
selected features: 184


In [22]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  0,   1,   2,   3,   4,  12,  13,  14,  20,  21,  27,  29,  36,
         39,  70,  74,  75,  87,  94, 100, 101, 102, 104, 116, 125, 135,
        138, 140, 146, 156, 161, 164, 166, 168, 177, 194, 196, 198, 199,
        200, 202, 204, 206, 208, 213, 220, 229, 231, 232, 233, 241, 242,
        243, 250, 265, 266, 270, 277, 280, 288, 296, 303, 307, 308, 310,
        316, 318, 319, 323, 325, 334, 338, 344, 345, 352, 357, 359, 363,
        367, 369, 374, 380, 386, 389, 390, 395, 396, 399, 410, 417, 428,
        433, 442, 444, 445, 447, 455, 457, 463, 466, 467, 468, 470, 471,
        473, 476, 477, 481, 482, 488, 490, 497, 503, 506, 515, 517, 518,
        519, 521, 534, 541, 544, 549, 553, 558, 559, 560, 561, 562, 563,
        564, 571, 573, 581, 585, 586, 588, 589, 593, 597, 602, 603, 604,
        605, 606, 608, 614, 617, 618, 619, 628, 634, 639, 651, 652, 670,
        673, 677, 678, 685, 702, 703, 704, 705, 712, 713, 716, 737, 738,
        742, 746, 750, 751, 757, 759, 766, 768, 779

In [23]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTTGTAATTTT', 'TTTTTTGTAATTTTT', 'TTTTTTATTTTGGAT',
       'TTTTTTATTTTGGATAA', 'TTTTTTATTTTGGATAAAAGGAG', 'TTTTCTTTTCGT',
       'TTTTCTTCTAATC', 'TTTTCTATTGTC',
       'TTTTATGGAAGGTAATTTTAAAAATGTAAAGAAGCTTAT',
       'TTTTATGGAAGGTAATTTTAAAAATGTAAAGAAGCTTATTTACGAAG',
       'TTTTAATAGCTAGCACTTAATTGTGTTGGCTATTTTTTATGTCCAAAACGTGCTGATGACATAAAAAGCACGCATGGAAAAACAGTCGACAGACTATAAA',
       'TTTGCCAGTATC', 'TTTCGCAAACTA', 'TTTCAGCGACT', 'TTGTGATGTTGTTT',
       'TTGGTTTTAAATTT', 'TTGGTTTTAAATTTTT', 'TTGATAAAGTTTA',
       'TTCTTTACATTTTTA', 'TTCTCTTCCATC', 'TTCTCTTCCATCCCTCATC',
       'TTCTCTTCCATCCCTCATCCTCCTC', 'TTCTATAAAAAGT', 'TTCATCGTCGA',
       'TTCAATCTAGAT', 'TTATTAGGTTCAAC', 'TTATCATCAAATG',
       'TTATAGTCTGTCGACTGTTTTTCCATGCGTGCTTTTTATGTCATCAGCACGTTTTGGACATAAAAAATAGCCAACACAATTAAGTGCTAGCTATTAAAAG',
       'TTAGGCGAAGAT', 'TTACGCAATAGTTTAGATGTAGA', 'TTACCTAAAAATAAAT',
       'TTAATTGAATAACGGGAAGTAGCTCAGCTTGGTAGAGCACTTGGTTTGGGACCAAGGGGTCGCAGGTTCGAATCCTGTCTTCCCGATTACTTC

In [24]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [25]:
df_sel['strain'] = X.iloc[:,0]

In [26]:
df_sel

,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTCTTTTCGT,TTTTCTTCTAATC,TTTTCTATTGTC,TTTTATGGAAGGTAATTTTAAAAATGTAAAGAAGCTTAT,TTTTATGGAAGGTAATTTTAAAAATGTAAAGAAGCTTATTTACGAAG,...,ATAGCGGAGTT,AGGTGTAATGCT,AGGGGTGATT,AGGCTAACTT,AGCATCTACTTTT,ACTGCGTTAGT,ACTAAATTCGT,AACCTAGAAAGTTT,pheno,strain
0,0,0,0,0,0,0,1,0,0,0,...,0,0,1,1,0,0,0,0,2,107
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,109
2,1,1,1,1,1,1,1,1,0,0,...,1,1,1,1,1,1,1,1,2,115
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2,120335
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2,SR4152
249,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,SR4153
250,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2,SR4155
251,1,1,1,1,1,1,1,1,0,0,...,1,0,1,1,1,1,1,1,2,SR4156


In [27]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 185) (253,) (253, 186)


In [28]:
df_sel['pheno'].value_counts()

2    181
1     47
0     25
Name: pheno, dtype: int64

In [29]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 181), (1, 181), (2, 181)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [30]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel_over)

In [31]:
dat5 = pd.DataFrame(X_sel_test_over[:,-1])
dat5['test'] = y_sel_test_over

In [32]:
dat5

,0,test
0,CFBRSa49,1
1,NRS108,2
2,MN105,2
3,CFBRSa03,2
4,BCH-SA-01,0
...,...,...
158,NRS027,0
159,BCH-SA-04,0
160,SR3585,2
161,504,1


In [33]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [83]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [84]:
model2_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 499us/step - loss: 1.1424 - accuracy: 0.3342 - val_loss: 1.0788 - val_accuracy: 0.4294
Epoch 2/100
380/380 [==============================] - 0s 81us/step - loss: 1.0708 - accuracy: 0.4368 - val_loss: 1.0371 - val_accuracy: 0.4724
Epoch 3/100
380/380 [==============================] - 0s 98us/step - loss: 1.0193 - accuracy: 0.4895 - val_loss: 1.0172 - val_accuracy: 0.4601
Epoch 4/100
380/380 [==============================] - 0s 101us/step - loss: 0.9796 - accuracy: 0.5158 - val_loss: 0.9844 - val_accuracy: 0.6012
Epoch 5/100
380/380 [==============================] - 0s 112us/step - loss: 0.9393 - accuracy: 0.5895 - val_loss: 0.9506 - val_accuracy: 0.5828
Epoch 6/100
380/380 [==============================] - 0s 105us/step - loss: 0.9007 - accuracy: 0.6632 - val_loss: 0.9176 - val_accuracy: 0.6442
Epoch 7/100
380/380 [==============================] - 0s 118us/step - loss: 0.8682 - 

380/380 [==============================] - 0s 119us/step - loss: 0.2486 - accuracy: 0.9132 - val_loss: 0.3731 - val_accuracy: 0.8405
Epoch 58/100
380/380 [==============================] - 0s 86us/step - loss: 0.2440 - accuracy: 0.9132 - val_loss: 0.3794 - val_accuracy: 0.8282
Epoch 59/100
380/380 [==============================] - 0s 104us/step - loss: 0.2447 - accuracy: 0.9105 - val_loss: 0.3716 - val_accuracy: 0.8528
Epoch 60/100
380/380 [==============================] - 0s 140us/step - loss: 0.2396 - accuracy: 0.9132 - val_loss: 0.3820 - val_accuracy: 0.8221
Epoch 61/100
380/380 [==============================] - 0s 114us/step - loss: 0.2337 - accuracy: 0.9158 - val_loss: 0.3524 - val_accuracy: 0.8528
Epoch 62/100
380/380 [==============================] - 0s 95us/step - loss: 0.2404 - accuracy: 0.9026 - val_loss: 0.3737 - val_accuracy: 0.8528
Epoch 63/100
380/380 [==============================] - 0s 99us/step - loss: 0.2416 - accuracy: 0.9158 - val_loss: 0.3529 - val_accuracy: 0

In [164]:
acc_test2_over = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over*100))

163/163 [==============================] - 0s 104us/step
over-sampling test accuracy: 84.05%


In [86]:
pred5 = model2_over.predict_classes(X_sel_test_over)
pred5

array([1, 2, 1, 2, 0, 1, 1, 2, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 2, 2, 1, 1,
       0, 2, 0, 1, 2, 1, 0, 2, 2, 0, 1, 0, 1, 1, 1, 0, 1, 2, 2, 2, 2, 1,
       0, 1, 1, 0, 1, 2, 1, 2, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 2, 2, 1, 2,
       1, 0, 2, 1, 2, 0, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 2, 1, 1, 0, 1, 0,
       0, 1, 2, 1, 1, 1, 2, 2, 0, 1, 0, 0, 1, 1, 1, 0, 2, 1, 2, 1, 1, 2,
       1, 0, 2, 1, 1, 0, 0, 1, 1, 2, 1, 0, 1, 0, 0, 1, 2, 0, 2, 1, 1, 2,
       1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 2, 2, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 0])

In [87]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,CFBRSa49,1,1
1,NRS108,2,2
2,MN105,2,1
3,CFBRSa03,2,2
4,BCH-SA-01,0,0
...,...,...,...
158,NRS027,0,0
159,BCH-SA-04,0,0
160,SR3585,2,1
161,504,1,1


In [88]:
proba5 = model2_over.predict_proba(X_sel_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [89]:
dat_proba5

,0,1,2
0,0.411915,0.469525,0.118560
1,0.000185,0.009521,0.990294
2,0.000036,0.863504,0.136460
3,0.000045,0.000156,0.999799
4,0.973129,0.000358,0.026513
...,...,...,...
158,0.996810,0.000148,0.003042
159,0.992484,0.004105,0.003411
160,0.262175,0.461844,0.275981
161,0.006449,0.981863,0.011687


In [90]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [91]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p11.csv", index = False,
         header=None)

In [168]:
hist2_over = model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 91us/step - loss: 0.2082 - accuracy: 0.9237 - val_loss: 0.3912 - val_accuracy: 0.8221
Epoch 2/100
380/380 [==============================] - 0s 106us/step - loss: 0.2081 - accuracy: 0.9158 - val_loss: 0.3695 - val_accuracy: 0.8405
Epoch 3/100
380/380 [==============================] - 0s 102us/step - loss: 0.2050 - accuracy: 0.9184 - val_loss: 0.3856 - val_accuracy: 0.8282
Epoch 4/100
380/380 [==============================] - 0s 72us/step - loss: 0.2001 - accuracy: 0.9263 - val_loss: 0.3812 - val_accuracy: 0.8221
Epoch 5/100
380/380 [==============================] - 0s 70us/step - loss: 0.1888 - accuracy: 0.9211 - val_loss: 0.3737 - val_accuracy: 0.8282
Epoch 6/100
380/380 [==============================] - 0s 75us/step - loss: 0.1957 - accuracy: 0.9263 - val_loss: 0.3579 - val_accuracy: 0.8282
Epoch 7/100
380/380 [==============================] - 0s 68us/step - loss: 0.2014 - acc

Epoch 57/100
380/380 [==============================] - 0s 125us/step - loss: 0.1653 - accuracy: 0.9289 - val_loss: 0.3880 - val_accuracy: 0.8282
Epoch 58/100
380/380 [==============================] - 0s 114us/step - loss: 0.1674 - accuracy: 0.9289 - val_loss: 0.3485 - val_accuracy: 0.8344
Epoch 59/100
380/380 [==============================] - 0s 87us/step - loss: 0.1740 - accuracy: 0.9289 - val_loss: 0.3870 - val_accuracy: 0.8282
Epoch 60/100
380/380 [==============================] - 0s 117us/step - loss: 0.1797 - accuracy: 0.9263 - val_loss: 0.3708 - val_accuracy: 0.8344
Epoch 61/100
380/380 [==============================] - 0s 89us/step - loss: 0.1767 - accuracy: 0.9263 - val_loss: 0.3672 - val_accuracy: 0.8344
Epoch 62/100
380/380 [==============================] - 0s 112us/step - loss: 0.1800 - accuracy: 0.9211 - val_loss: 0.3672 - val_accuracy: 0.8282
Epoch 63/100
380/380 [==============================] - 0s 96us/step - loss: 0.1688 - accuracy: 0.9263 - val_loss: 0.3678 - va

In [169]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over.history['accuracy'])*100))

over-sampling train accuracy: 92.29%


In [34]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [35]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS245,1,2,1.345807e-02,2.164788e-01,7.700630e-01
1,p0006kpresabs_qual,NY439,2,2,2.674153e-02,9.294230e-04,9.723290e-01
2,p0006kpresabs_qual,CA544,1,0,4.147484e-01,3.626331e-01,2.226184e-01
3,p0006kpresabs_qual,CA541,2,0,4.147484e-01,3.626331e-01,2.226184e-01
4,p0006kpresabs_qual,EUH15,1,0,4.147484e-01,3.626331e-01,2.226184e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,CA541,1,1,3.723218e-01,6.276781e-01,1.945911e-08
985,p0017Skpresabs_qual,SR4152,1,0,7.372800e-01,2.627200e-01,4.197748e-08
986,p0017Skpresabs_qual,NRS110,2,2,4.194510e-08,7.508231e-09,1.000000e+00
987,p0017Skpresabs_qual,CFBRSa70,0,0,7.372800e-01,2.627200e-01,4.197748e-08


In [36]:
y_prob5 = df_proba5[df_proba5['phage']=='p11kpresabs_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[4.11915480e-01, 4.69524650e-01, 1.18559930e-01],
       [1.85266310e-04, 9.52110300e-03, 9.90293600e-01],
       [3.60156920e-05, 8.63503800e-01, 1.36460160e-01],
       [4.47145980e-05, 1.55991150e-04, 9.99799300e-01],
       [9.73128600e-01, 3.57974520e-04, 2.65133900e-02],
       [2.07474390e-06, 9.99939440e-01, 5.85182750e-05],
       [4.47925930e-03, 6.53110300e-01, 3.42410360e-01],
       [8.66245200e-06, 7.00983870e-03, 9.92981430e-01],
       [9.93974600e-01, 4.42929500e-03, 1.59612850e-03],
       [8.00493540e-01, 5.56601730e-02, 1.43846350e-01],
       [8.58791900e-06, 9.76473100e-01, 2.35182720e-02],
       [4.17082200e-05, 9.99937800e-01, 2.04926220e-05],
       [4.11915480e-01, 4.69524650e-01, 1.18559930e-01],
       [1.82089920e-01, 7.36610100e-01, 8.12999400e-02],
       [9.97834600e-01, 2.91613160e-04, 1.87375400e-03],
       [6.53518140e-01, 2.28519260e-01, 1.17962636e-01],
       [4.75453040e-04, 9.98560850e-01, 9.63785100e-04],
       [6.61089600e-01, 2.46374

In [37]:
ovo5 = rocauc_ovo(y_sel_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.9762062521236835

In [38]:
ovr5 = rocauc_ovr(y_sel_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.9762062521236835

In [39]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel_over)

In [40]:
dat6 = pd.DataFrame(X_sel_test_over[:,-1])
dat6['test'] = y_sel_test_over

In [41]:
dat6

,0,test
0,GA48963,1
1,SR4187,2
2,NRS182,2
3,CFBREBSa125,2
4,NRS188,1
...,...,...
158,BCH-SA-05,0
159,NRS027,0
160,CFBREBSa123,0
161,NRS199,2


In [42]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [96]:
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [97]:
model2_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 315us/step - loss: 1.1623 - accuracy: 0.3737 - val_loss: 1.1773 - val_accuracy: 0.3313
Epoch 2/100
380/380 [==============================] - 0s 95us/step - loss: 1.0969 - accuracy: 0.4184 - val_loss: 1.0819 - val_accuracy: 0.4049
Epoch 3/100
380/380 [==============================] - 0s 61us/step - loss: 1.0238 - accuracy: 0.5000 - val_loss: 1.0314 - val_accuracy: 0.5276
Epoch 4/100
380/380 [==============================] - 0s 76us/step - loss: 0.9774 - accuracy: 0.5579 - val_loss: 1.0038 - val_accuracy: 0.4663
Epoch 5/100
380/380 [==============================] - 0s 100us/step - loss: 0.9556 - accuracy: 0.6184 - val_loss: 0.9948 - val_accuracy: 0.4908
Epoch 6/100
380/380 [==============================] - 0s 111us/step - loss: 0.9206 - accuracy: 0.6211 - val_loss: 0.9570 - val_accuracy: 0.5583
Epoch 7/100
380/380 [==============================] - 0s 95us/step - loss: 0.8817 - ac

380/380 [==============================] - 0s 138us/step - loss: 0.3607 - accuracy: 0.8684 - val_loss: 0.5243 - val_accuracy: 0.7791
Epoch 58/100
380/380 [==============================] - 0s 121us/step - loss: 0.3602 - accuracy: 0.8579 - val_loss: 0.5192 - val_accuracy: 0.7975
Epoch 59/100
380/380 [==============================] - 0s 130us/step - loss: 0.3513 - accuracy: 0.8842 - val_loss: 0.5273 - val_accuracy: 0.7607
Epoch 60/100
380/380 [==============================] - 0s 120us/step - loss: 0.3525 - accuracy: 0.8789 - val_loss: 0.5182 - val_accuracy: 0.7730
Epoch 61/100
380/380 [==============================] - 0s 96us/step - loss: 0.3491 - accuracy: 0.8842 - val_loss: 0.5152 - val_accuracy: 0.7546
Epoch 62/100
380/380 [==============================] - 0s 80us/step - loss: 0.3462 - accuracy: 0.8684 - val_loss: 0.5048 - val_accuracy: 0.7914
Epoch 63/100
380/380 [==============================] - 0s 63us/step - loss: 0.3461 - accuracy: 0.8711 - val_loss: 0.5033 - val_accuracy: 0

In [210]:
acc_test2_over2 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over2*100))

ValueError: Error when checking input: expected dense_69_input to have shape (183,) but got array with shape (184,)

In [99]:
pred6 = model2_over2.predict_classes(X_sel_test_over)
pred6

array([1, 2, 2, 1, 1, 0, 1, 2, 2, 1, 2, 1, 0, 0, 1, 0, 2, 1, 2, 1, 2, 1,
       0, 1, 1, 2, 0, 2, 0, 2, 0, 0, 0, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 0,
       1, 2, 2, 2, 0, 1, 2, 1, 2, 2, 0, 0, 1, 2, 0, 0, 1, 1, 2, 1, 0, 1,
       2, 2, 0, 2, 2, 1, 1, 1, 1, 2, 2, 1, 0, 1, 2, 1, 2, 1, 1, 2, 1, 1,
       2, 1, 1, 2, 1, 0, 2, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       2, 2, 2, 2, 0, 2, 1, 0, 1, 0, 1, 0, 2, 0, 2, 0, 0, 1, 2, 1, 0, 1,
       1, 1, 2, 2, 2, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 2, 1, 0, 0, 2, 0, 1,
       2, 1, 0, 0, 0, 0, 0, 2, 0])

In [100]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,GA48963,1,1
1,SR4187,2,2
2,NRS182,2,2
3,CFBREBSa125,2,1
4,NRS188,1,1
...,...,...,...
158,BCH-SA-05,0,0
159,NRS027,0,0
160,CFBREBSa123,0,0
161,NRS199,2,2


In [101]:
proba6 = model2_over2.predict_proba(X_sel_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [102]:
dat_proba6

,0,1,2
0,0.002574,0.885729,0.111697
1,0.118700,0.210628,0.670673
2,0.043093,0.105198,0.851708
3,0.245581,0.651203,0.103216
4,0.000436,0.977856,0.021708
...,...,...,...
158,0.962178,0.013605,0.024217
159,0.983021,0.005792,0.011186
160,0.655458,0.002887,0.341655
161,0.013677,0.003953,0.982369


In [103]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [104]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p11.csv", index = False,
         header=None)

In [197]:
hist2_over2 = model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 82us/step - loss: 0.2749 - accuracy: 0.9105 - val_loss: 0.4064 - val_accuracy: 0.8650
Epoch 2/100
380/380 [==============================] - 0s 97us/step - loss: 0.2704 - accuracy: 0.9079 - val_loss: 0.4095 - val_accuracy: 0.8650
Epoch 3/100
380/380 [==============================] - 0s 100us/step - loss: 0.2688 - accuracy: 0.9079 - val_loss: 0.4106 - val_accuracy: 0.8528
Epoch 4/100
380/380 [==============================] - 0s 85us/step - loss: 0.2752 - accuracy: 0.9079 - val_loss: 0.4065 - val_accuracy: 0.8650
Epoch 5/100
380/380 [==============================] - 0s 72us/step - loss: 0.2729 - accuracy: 0.9053 - val_loss: 0.3993 - val_accuracy: 0.8650
Epoch 6/100
380/380 [==============================] - 0s 68us/step - loss: 0.2643 - accuracy: 0.9079 - val_loss: 0.4035 - val_accuracy: 0.8650
Epoch 7/100
380/380 [==============================] - 0s 71us/step - loss: 0.2675 - accu

In [198]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over2.history['accuracy'])*100))

over-sampling train accuracy: 91.57%


In [43]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [44]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS249,2,1,1.888869e-01,5.108038e-01,3.003094e-01
1,p0006kpresabs_qual,NRS188,1,1,1.888869e-01,5.108038e-01,3.003094e-01
2,p0006kpresabs_qual,NRS232,2,2,4.222906e-01,7.029924e-02,5.074101e-01
3,p0006kpresabs_qual,NY439,2,2,3.558408e-04,2.976018e-04,9.993465e-01
4,p0006kpresabs_qual,GA27,2,1,3.940971e-01,4.184215e-01,1.874814e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,NRS252,0,0,7.239556e-01,2.760444e-01,1.176030e-09
985,p0017Skpresabs_qual,SR2852,1,1,1.052276e-07,9.999999e-01,1.101559e-28
986,p0017Skpresabs_qual,NRS108,1,1,1.540350e-17,1.000000e+00,9.011977e-16
987,p0017Skpresabs_qual,NRS202,0,0,6.888959e-01,3.111042e-01,2.228958e-09


In [45]:
y_prob6 = df_proba6[df_proba6['phage']=='p11kpresabs_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[2.57378890e-03, 8.85729100e-01, 1.11697110e-01],
       [1.18699506e-01, 2.10627940e-01, 6.70672540e-01],
       [4.30934470e-02, 1.05198040e-01, 8.51708500e-01],
       [2.45580930e-01, 6.51203200e-01, 1.03215870e-01],
       [4.35716620e-04, 9.77855860e-01, 2.17083600e-02],
       [9.78418700e-01, 5.73518870e-03, 1.58461430e-02],
       [7.86796000e-03, 9.69179600e-01, 2.29524260e-02],
       [3.83677870e-01, 1.22634664e-01, 4.93687500e-01],
       [2.77630700e-03, 5.93725100e-02, 9.37851200e-01],
       [2.20101040e-01, 5.52219100e-01, 2.27679920e-01],
       [3.53825270e-01, 1.98377700e-01, 4.47797030e-01],
       [6.19283280e-02, 5.99709000e-01, 3.38362660e-01],
       [5.54868900e-01, 2.27175970e-01, 2.17955100e-01],
       [8.17905070e-01, 7.51824160e-02, 1.06912486e-01],
       [2.45580930e-01, 6.51203200e-01, 1.03215870e-01],
       [8.56446150e-01, 2.02935190e-04, 1.43350880e-01],
       [2.70295090e-02, 1.55329500e-02, 9.57437500e-01],
       [5.89528900e-04, 9.39687

In [46]:
ovo6 = rocauc_ovo(y_sel_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.9506739155057198

In [47]:
ovr6 = rocauc_ovr(y_sel_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.9506739155057198

In [48]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel_over)

In [49]:
dat7 = pd.DataFrame(X_sel_test_over[:,-1])
dat7['test'] = y_sel_test_over

In [50]:
dat7

,0,test
0,CFBREBSa127,1
1,NRS145,0
2,CFBRSa66B,1
3,NRS204,1
4,BCH-SA-13,2
...,...,...
158,NRS233,2
159,NRS204,1
160,CFBRSa07,0
161,CFBREBSa117,1


In [51]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [109]:
model2_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [110]:
model2_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model2_over3.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 396us/step - loss: 1.1924 - accuracy: 0.3421 - val_loss: 1.1260 - val_accuracy: 0.3742
Epoch 2/100
380/380 [==============================] - 0s 88us/step - loss: 1.0327 - accuracy: 0.4237 - val_loss: 1.0526 - val_accuracy: 0.4356
Epoch 3/100
380/380 [==============================] - 0s 132us/step - loss: 0.9660 - accuracy: 0.5500 - val_loss: 0.9896 - val_accuracy: 0.4847
Epoch 4/100
380/380 [==============================] - 0s 124us/step - loss: 0.9188 - accuracy: 0.5684 - val_loss: 0.9463 - val_accuracy: 0.5215
Epoch 5/100
380/380 [==============================] - 0s 155us/step - loss: 0.8724 - accuracy: 0.6211 - val_loss: 0.9104 - val_accuracy: 0.5521
Epoch 6/100
380/380 [==============================] - 0s 149us/step - loss: 0.8275 - accuracy: 0.6579 - val_loss: 0.8671 - val_accuracy: 0.6135
Epoch 7/100
380/380 [==============================] - 0s 111us/step - loss: 0.7933 -

Epoch 57/100
380/380 [==============================] - 0s 209us/step - loss: 0.2818 - accuracy: 0.8947 - val_loss: 0.4449 - val_accuracy: 0.8282
Epoch 58/100
380/380 [==============================] - 0s 213us/step - loss: 0.2861 - accuracy: 0.8947 - val_loss: 0.4383 - val_accuracy: 0.8282
Epoch 59/100
380/380 [==============================] - 0s 217us/step - loss: 0.2787 - accuracy: 0.8947 - val_loss: 0.4691 - val_accuracy: 0.8037
Epoch 60/100
380/380 [==============================] - 0s 171us/step - loss: 0.2934 - accuracy: 0.8842 - val_loss: 0.4483 - val_accuracy: 0.8344
Epoch 61/100
380/380 [==============================] - 0s 217us/step - loss: 0.2807 - accuracy: 0.9053 - val_loss: 0.4612 - val_accuracy: 0.8528
Epoch 62/100
380/380 [==============================] - 0s 186us/step - loss: 0.2700 - accuracy: 0.8921 - val_loss: 0.4488 - val_accuracy: 0.8528
Epoch 63/100
380/380 [==============================] - 0s 126us/step - loss: 0.2764 - accuracy: 0.8974 - val_loss: 0.4456 -

In [263]:
acc_test2_over3 = model2_over3.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over3*100))

163/163 [==============================] - 0s 83us/step
over-sampling test accuracy: 83.44%


In [112]:
pred7 = model2_over3.predict_classes(X_sel_test_over)
pred7

array([1, 0, 1, 1, 1, 0, 0, 0, 2, 1, 1, 1, 2, 2, 0, 2, 0, 1, 2, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 2, 1, 2, 2, 0, 1, 2, 2, 1, 0, 1, 2, 0, 2, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 2, 0, 1, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0,
       1, 2, 1, 1, 1, 2, 1, 1, 0, 1, 1, 2, 0, 0, 1, 2, 2, 0, 2, 0, 2, 0,
       0, 0, 1, 1, 2, 2, 2, 2, 1, 0, 2, 0, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2,
       0, 0, 0, 1, 2, 0, 0, 1, 0, 2, 2, 2, 0, 1, 1, 1, 1, 2, 0, 2, 1, 0,
       2, 1, 1, 0, 0, 0, 1, 1, 1, 2, 0, 1, 2, 2, 0, 1, 1, 2, 1, 1, 0, 1,
       0, 0, 0, 2, 2, 1, 0, 1, 0])

In [113]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,CFBREBSa127,1,1
1,NRS145,0,0
2,CFBRSa66B,1,1
3,NRS204,1,1
4,BCH-SA-13,2,1
...,...,...,...
158,NRS233,2,2
159,NRS204,1,1
160,CFBRSa07,0,0
161,CFBREBSa117,1,1


In [114]:
proba7 = model2_over3.predict_proba(X_sel_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [115]:
dat_proba7

,0,1,2
0,0.041830,0.945889,0.012282
1,0.979797,0.015503,0.004700
2,0.203042,0.520154,0.276803
3,0.001634,0.984215,0.014151
4,0.314982,0.621410,0.063608
...,...,...,...
158,0.000005,0.000161,0.999834
159,0.001634,0.984215,0.014151
160,0.954545,0.000749,0.044706
161,0.228832,0.663252,0.107916


In [116]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [117]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p11.csv", index = False,
         header=None)

In [267]:
hist2_over3 = model2_over3.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 105us/step - loss: 0.2186 - accuracy: 0.9132 - val_loss: 0.4422 - val_accuracy: 0.8466
Epoch 2/100
380/380 [==============================] - 0s 121us/step - loss: 0.2107 - accuracy: 0.9263 - val_loss: 0.4208 - val_accuracy: 0.8466
Epoch 3/100
380/380 [==============================] - 0s 89us/step - loss: 0.2118 - accuracy: 0.9132 - val_loss: 0.4142 - val_accuracy: 0.8466
Epoch 4/100
380/380 [==============================] - 0s 87us/step - loss: 0.2111 - accuracy: 0.9237 - val_loss: 0.4230 - val_accuracy: 0.8466
Epoch 5/100
380/380 [==============================] - 0s 87us/step - loss: 0.2079 - accuracy: 0.9211 - val_loss: 0.4145 - val_accuracy: 0.8466
Epoch 6/100
380/380 [==============================] - 0s 83us/step - loss: 0.2124 - accuracy: 0.9237 - val_loss: 0.4484 - val_accuracy: 0.8466
Epoch 7/100
380/380 [==============================] - 0s 88us/step - loss: 0.2164 - acc

In [268]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over3.history['accuracy'])*100))

over-sampling train accuracy: 92.02%


In [52]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [53]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS210,0,0,6.132076e-01,2.812180e-01,1.055744e-01
1,p0006kpresabs_qual,NRS205,2,2,1.993202e-04,6.834937e-07,9.998000e-01
2,p0006kpresabs_qual,312,2,1,3.589463e-01,3.982787e-01,2.427750e-01
3,p0006kpresabs_qual,GA15,2,1,3.589463e-01,3.982787e-01,2.427750e-01
4,p0006kpresabs_qual,SR4035,0,1,3.589463e-01,3.982787e-01,2.427750e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,NRS383,1,0,5.477194e-01,4.522807e-01,1.761374e-08
985,p0017Skpresabs_qual,NRS218,1,1,6.953657e-05,9.999305e-01,3.132419e-10
986,p0017Skpresabs_qual,NRS209,2,2,2.713214e-09,6.656316e-09,1.000000e+00
987,p0017Skpresabs_qual,SR2852,1,1,9.956684e-12,1.000000e+00,7.441288e-26


In [54]:
y_prob7 = df_proba7[df_proba7['phage']=='p11kpresabs_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[4.18298060e-02, 9.45888700e-01, 1.22816170e-02],
       [9.79796950e-01, 1.55026970e-02, 4.70028770e-03],
       [2.03042490e-01, 5.20154400e-01, 2.76803140e-01],
       [1.63378920e-03, 9.84215440e-01, 1.41508020e-02],
       [3.14981760e-01, 6.21409800e-01, 6.36084750e-02],
       [9.92449300e-01, 3.22524180e-03, 4.32556080e-03],
       [9.92570600e-01, 2.82636380e-03, 4.60304550e-03],
       [9.98242000e-01, 1.65318630e-03, 1.04834170e-04],
       [3.31534670e-02, 4.17398780e-01, 5.49447800e-01],
       [2.28831710e-01, 6.63252230e-01, 1.07916080e-01],
       [4.34379730e-02, 9.50388600e-01, 6.17349800e-03],
       [4.18298060e-02, 9.45888700e-01, 1.22816170e-02],
       [2.45814490e-01, 1.75794840e-01, 5.78390700e-01],
       [2.69555630e-04, 2.42486040e-03, 9.97305630e-01],
       [6.24578830e-01, 2.10758240e-01, 1.64662930e-01],
       [1.10714085e-01, 4.16145800e-01, 4.73140120e-01],
       [9.51160700e-01, 4.37449600e-02, 5.09426560e-03],
       [4.51731350e-04, 9.97174

In [55]:
ovo7 = rocauc_ovo(y_sel_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.9639063931877389

In [56]:
ovr7 = rocauc_ovr(y_sel_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.9639063931877389

In [57]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel_over)

In [58]:
dat8 = pd.DataFrame(X_sel_test_over[:,-1])
dat8['test'] = y_sel_test_over

In [59]:
dat8

,0,test
0,SR2852,2
1,NRS054,1
2,NRS157,2
3,NY224,1
4,NRS070,1
...,...,...
158,NY417,2
159,NRS051,1
160,NRS226,1
161,EUH13,0


In [60]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [100]:
model2_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [123]:
model2_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [124]:
model2_over4.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 394us/step - loss: 1.1455 - accuracy: 0.3579 - val_loss: 1.1096 - val_accuracy: 0.3742
Epoch 2/100
380/380 [==============================] - 0s 158us/step - loss: 1.0360 - accuracy: 0.4737 - val_loss: 1.0614 - val_accuracy: 0.4356
Epoch 3/100
380/380 [==============================] - 0s 76us/step - loss: 0.9965 - accuracy: 0.5737 - val_loss: 1.0365 - val_accuracy: 0.4356
Epoch 4/100
380/380 [==============================] - 0s 128us/step - loss: 0.9747 - accuracy: 0.5737 - val_loss: 1.0264 - val_accuracy: 0.4479
Epoch 5/100
380/380 [==============================] - 0s 95us/step - loss: 0.9371 - accuracy: 0.5605 - val_loss: 0.9960 - val_accuracy: 0.5031
Epoch 6/100
380/380 [==============================] - 0s 111us/step - loss: 0.9115 - accuracy: 0.6237 - val_loss: 0.9766 - val_accuracy: 0.4785
Epoch 7/100
380/380 [==============================] - 0s 65us/step - loss: 0.8882 - a

380/380 [==============================] - 0s 221us/step - loss: 0.3285 - accuracy: 0.8921 - val_loss: 0.5479 - val_accuracy: 0.7607
Epoch 58/100
380/380 [==============================] - 0s 270us/step - loss: 0.3305 - accuracy: 0.8816 - val_loss: 0.5413 - val_accuracy: 0.7730
Epoch 59/100
380/380 [==============================] - 0s 309us/step - loss: 0.3220 - accuracy: 0.8921 - val_loss: 0.5453 - val_accuracy: 0.7607
Epoch 60/100
380/380 [==============================] - 0s 112us/step - loss: 0.3136 - accuracy: 0.9105 - val_loss: 0.5353 - val_accuracy: 0.7607
Epoch 61/100
380/380 [==============================] - 0s 78us/step - loss: 0.3102 - accuracy: 0.8974 - val_loss: 0.5453 - val_accuracy: 0.7730
Epoch 62/100
380/380 [==============================] - 0s 64us/step - loss: 0.3090 - accuracy: 0.8947 - val_loss: 0.5368 - val_accuracy: 0.7730
Epoch 63/100
380/380 [==============================] - 0s 62us/step - loss: 0.2996 - accuracy: 0.9158 - val_loss: 0.5304 - val_accuracy: 0

In [296]:
acc_test2_over4 = model2_over4.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over4*100))

163/163 [==============================] - 0s 99us/step
over-sampling test accuracy: 80.37%


In [125]:
pred8 = model2_over4.predict_classes(X_sel_test_over)
pred8

array([2, 1, 2, 2, 1, 1, 2, 2, 1, 0, 2, 2, 2, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       2, 2, 0, 2, 2, 2, 2, 1, 0, 1, 2, 2, 1, 1, 0, 2, 1, 0, 2, 1, 1, 0,
       0, 2, 0, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 0, 2, 1, 0, 0, 0,
       0, 1, 1, 0, 2, 0, 1, 1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 2, 2, 1, 2, 2,
       0, 2, 0, 0, 0, 2, 2, 1, 1, 0, 1, 1, 1, 2, 0, 2, 1, 2, 2, 0, 0, 2,
       1, 1, 1, 0, 0, 2, 1, 1, 1, 2, 1, 2, 2, 0, 2, 2, 0, 0, 1, 1, 2, 0,
       0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 2, 0, 0,
       2, 2, 2, 2, 2, 1, 1, 0, 1])

In [126]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,SR2852,2,2
1,NRS054,1,1
2,NRS157,2,2
3,NY224,1,2
4,NRS070,1,1
...,...,...,...
158,NY417,2,2
159,NRS051,1,1
160,NRS226,1,1
161,EUH13,0,0


In [127]:
proba8 = model2_over4.predict_proba(X_sel_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [128]:
dat_proba8

,0,1,2
0,0.002746,0.288469,0.708786
1,0.004718,0.975754,0.019529
2,0.000192,0.007352,0.992455
3,0.058554,0.185173,0.756274
4,0.000289,0.963804,0.035907
...,...,...,...
158,0.058554,0.185173,0.756274
159,0.007984,0.972268,0.019748
160,0.003023,0.756889,0.240087
161,0.989564,0.002574,0.007862


In [129]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [130]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p11.csv", index = False,
         header=None)

In [300]:
hist2_over4 = model2_over4.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 380 samples, validate on 163 samples
Epoch 1/100
380/380 [==============================] - 0s 146us/step - loss: 0.2564 - accuracy: 0.9053 - val_loss: 0.5488 - val_accuracy: 0.8037
Epoch 2/100
380/380 [==============================] - 0s 145us/step - loss: 0.2737 - accuracy: 0.8711 - val_loss: 0.6301 - val_accuracy: 0.7485
Epoch 3/100
380/380 [==============================] - 0s 127us/step - loss: 0.2742 - accuracy: 0.8895 - val_loss: 0.5938 - val_accuracy: 0.7914
Epoch 4/100
380/380 [==============================] - 0s 130us/step - loss: 0.2369 - accuracy: 0.9132 - val_loss: 0.5442 - val_accuracy: 0.7853
Epoch 5/100
380/380 [==============================] - 0s 118us/step - loss: 0.2305 - accuracy: 0.9184 - val_loss: 0.5575 - val_accuracy: 0.8282
Epoch 6/100
380/380 [==============================] - 0s 138us/step - loss: 0.2543 - accuracy: 0.8763 - val_loss: 0.5614 - val_accuracy: 0.7669
Epoch 7/100
380/380 [==============================] - 0s 110us/step - loss: 0.2188 

Epoch 57/100
380/380 [==============================] - 0s 83us/step - loss: 0.2006 - accuracy: 0.9184 - val_loss: 0.5456 - val_accuracy: 0.7791
Epoch 58/100
380/380 [==============================] - 0s 98us/step - loss: 0.1853 - accuracy: 0.9316 - val_loss: 0.5438 - val_accuracy: 0.7730
Epoch 59/100
380/380 [==============================] - 0s 85us/step - loss: 0.1803 - accuracy: 0.9316 - val_loss: 0.6363 - val_accuracy: 0.8037
Epoch 60/100
380/380 [==============================] - 0s 103us/step - loss: 0.1863 - accuracy: 0.9079 - val_loss: 0.5529 - val_accuracy: 0.8344
Epoch 61/100
380/380 [==============================] - 0s 101us/step - loss: 0.1848 - accuracy: 0.9289 - val_loss: 0.5541 - val_accuracy: 0.7853
Epoch 62/100
380/380 [==============================] - 0s 87us/step - loss: 0.1986 - accuracy: 0.9211 - val_loss: 0.5683 - val_accuracy: 0.8160
Epoch 63/100
380/380 [==============================] - 0s 95us/step - loss: 0.1724 - accuracy: 0.9316 - val_loss: 0.5503 - val_

In [301]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over4.history['accuracy'])*100))

over-sampling train accuracy: 91.86%


In [61]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [62]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS236,1,2,1.321970e-02,2.446264e-01,7.421539e-01
1,p0006kpresabs_qual,NRS113,2,2,3.478230e-02,2.806685e-01,6.845492e-01
2,p0006kpresabs_qual,CFBRSa23,0,0,4.090251e-01,3.405008e-01,2.504741e-01
3,p0006kpresabs_qual,NRS249,2,1,1.987907e-01,5.331044e-01,2.681049e-01
4,p0006kpresabs_qual,107,1,0,4.090251e-01,3.405008e-01,2.504741e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,CFBRSa30,0,0,7.207667e-01,2.792331e-01,2.571588e-07
985,p0017Skpresabs_qual,NRS383,1,0,6.129044e-01,3.870795e-01,1.601290e-05
986,p0017Skpresabs_qual,NRS110,2,2,3.260306e-07,7.910664e-07,9.999989e-01
987,p0017Skpresabs_qual,NRS209,2,2,3.604249e-12,2.698129e-07,9.999998e-01


In [63]:
y_prob8 = df_proba8[df_proba8['phage']=='p11kpresabs_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[2.74581700e-03, 2.88468540e-01, 7.08785600e-01],
       [4.71758560e-03, 9.75753660e-01, 1.95287750e-02],
       [1.92255250e-04, 7.35249100e-03, 9.92455240e-01],
       [5.85535060e-02, 1.85172830e-01, 7.56273700e-01],
       [2.88878950e-04, 9.63804100e-01, 3.59069700e-02],
       [2.23829400e-06, 9.92302660e-01, 7.69512730e-03],
       [1.25340930e-03, 3.29659900e-02, 9.65780600e-01],
       [3.12293350e-01, 5.72142100e-02, 6.30492400e-01],
       [6.43630160e-04, 9.81170500e-01, 1.81859140e-02],
       [9.37851700e-01, 4.23572140e-02, 1.97910460e-02],
       [3.11034360e-02, 2.67407200e-01, 7.01489300e-01],
       [7.15927700e-03, 8.03341300e-03, 9.84807250e-01],
       [1.73330750e-01, 2.99731760e-01, 5.26937500e-01],
       [4.71758560e-03, 9.75753660e-01, 1.95287750e-02],
       [1.47159160e-01, 5.85265000e-01, 2.67575830e-01],
       [9.03133870e-01, 5.17746880e-02, 4.50914540e-02],
       [9.88825000e-01, 7.81643200e-05, 1.10968060e-02],
       [9.98544200e-04, 9.04473

In [64]:
ovo8 = rocauc_ovo(y_sel_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.9384924680031714

In [65]:
ovr8 = rocauc_ovr(y_sel_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.9384924680031714

In [66]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.9573197572050784

In [67]:
np.std(ovos2)

0.014130795222348162

In [68]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.9573197572050784

In [69]:
np.std(ovrs2)

0.014130795222348162

In [302]:
accs_l_over = [acc_test2_over, acc_test2_over2, acc_test2_over3, acc_test2_over4]

In [303]:
mean_l_over = np.mean(accs_l_over)
print('over-sampling test accuracy mean after lasso: %.2f%%' % (mean_l_over*100))

over-sampling test accuracy mean after lasso: 83.44%


In [304]:
std_l_over = np.std(accs_l_over)
print('over-sampling test accuracy standard deviation after lasso:', std_l_over)

over-sampling test accuracy standard deviation after lasso: 0.019879588183034303


In [305]:
accs_train_l_over = [np.mean(hist2_over.history['accuracy']), np.mean(hist2_over2.history['accuracy']), np.mean(hist2_over3.history['accuracy']),
             np.mean(hist2_over4.history['accuracy'])]

In [306]:
mean_train_l_over = np.mean(accs_train_l_over)
print('over-sampling train accuracy mean after lasso: %.2f%%' % (mean_train_l_over*100))

over-sampling train accuracy mean after lasso: 91.94%


In [307]:
std_train_l_over = np.std(accs_train_l_over)
print('over-sampling train accuracy standard deviation after lasso:', std_train_l_over)

over-sampling train accuracy standard deviation after lasso: 0.0026263413
